

<img src = 'https://t1.daumcdn.net/thumb/R1280x0/?fname=http://t1.daumcdn.net/brunch/service/user/11Yx/image/jNo1MfbPhIsud4DhAFh5fej9nQA.jpg' width=180 />
    
<center><small><a href="https://kmong.com/">kmong.com</a></small></center>

주어진 데이터를 활용하여 사용자에게 새로운 상품을 추천하거나,오픈마켓에서 금기시되는 개인간 거래를 발견하는등 다양한 분야에서 데이터와 머신러닝 알고리즘을 사용. 여기에는 크몽의 서비스에서 발생한 다양한 데이터를 사용하게 되는데, 이 부분에서 가장 중요한 것이 바로 데이터를 정리하는 과정, 일명 데이터 클리닝([Data Cleaning](https://en.wikipedia.org/wiki/Data_cleansing))이다.



In [2]:
import pandas as pd
pd.options.display.max_columns = 100


**kmong-conversion.csv**

가장 처음 가져올 데이터는 크몽의 가장 핵심 데이터라고 할 수 있는 ```kmong-conversion.csv```이다. 여기에는 사용자가 크몽의 웹사이트/모바일 서비스에서 활동한 모든 활동 기록(activity)이 상세하게 담겨져 있으며, 이 정보를 바탕으로 데이터를 분석하여 사용자의 패턴을 유추할 수 있다.

* ```eventcategory```: 앱의 이벤트에 대한 카테고리
  - ```install```: 앱설치
  - ```launch```: 앱실행 
  - ```deeplinkLaunch```: 딥링크 통한 앱실행
  - ```goal```: 앱 내에서의 활동 여부
  - ```exit```: 앱 종료
  - ```foreground```, background```: 앱을 foreground, background로 전환
  - ```launchlnSession```: Session의 시작.
* ```isfirstactivity```: 해당 이벤트가 해당 유저에 한하여 첫 번째인지 혹은 두 번째 이상인지를 알려주는 값. 
  - boolean 형식으로, 해당 이벤트가 해당 유저에 한하여 첫 번째 이벤트이면 True, 아니면 False를 갖는다.
* ```apppackagename```: 앱의 패키지의 고유한 이름. 안드로이드의 경우 applicationId을, iOS의 경우 Bundle ID를 가져옴.
* ```appversion```: kmong 어플리케이션의 버전.
* ```devicetype```: 사용자가 kmong app을 실행하고 있는 기기의 제품명.
* ```devicemanufacturer```: 사용자가 kmong app을 실행하고 있는 기기의 제조회사. 
* ```osversion```: 사용자가 kmong app을 실행하고 있는 기기의 os 버전.
* ```canonicaldeviceuuid```: 사용자가 kmong app을 실행하고 있는 기기의 고유 식별자. (사용자를 구분하는 식별자로 이용할 수 있다)
* ```sourcetype```: 해당 로그를 만든 고객이 유입된 경로. 유료 광고 채널, 바이럴 채널, 앱 마켓 등으로 구분.
* ```channel```: sourcetype과 동일하지만 조금 더 세분화.
  - unattributed, WEB, google-play, m_naver, google, (not set), google.adwords, m_naverpowercontents, pc_naver, apple.searchads, facebook, m_daum
* ```params_campaign```: 마케터가 입력한 캠페인 파라미터 중 캠페인. 
* ```params_medium```: 마케터가 입력한 캠페인 파라미터 중 매체.
* ```params_term```: 마케터가 입력한 캠페인 파라미터 중 키워드.
* ```inappeventcategory```: 인앱이벤트 식별자값으로 인앱이벤트를 지칭하는 3개의 값(category > action > label) 중 가장 상위에 속하는 구분값.
  - ```eventcategory```가 goal인 경우에만 제공.
  - 차후에 읽어올 funnel 데이터셋과 연결할 수 있다.
  - ex) seller_selling_history.view, gig_detail.view
* ```inappeventlabel```: categoryid와 동일.
  - 차후에 읽어올 category 데이터셋과 연결.
  - 인앱이벤트를 지칭하는 3개의 값(category > action > label) 중 마지막에 속하는 구분값.
* ```eventdatetime```: 모바일 클라이언트 상에서의 실제 이벤트 발생시. ISO 8601 표준에 따라서 제공. 
* ```isfirstgoalactivity```: goal 이벤트의 경우, 해당 Goal Label, Description, Key, Category를 가진 이벤트가 해당 유저에 한하여 첫 번째인지 혹은 두 번째 이상 인지를 알려주는 boolean 값. 예를 들어 동일한 유저로부터 두 번 이상의 같은 Goal 이벤트가 발생한 경우(e.g. 동일한 물건 재구매 등) 이 값은 false.


In [3]:

raw_log = pd.read_csv("./kmong/data/kmong-conversion.csv")

print(raw_log.shape)

raw_log.head()

(434244, 19)


,eventcategory,isfirstactivity,apppackagename,appversion,devicetype,devicemanufacturer,osversion,canonicaldeviceuuid,sourcetype,channel,params_campaign,params_medium,params_term,inappeventcategory,inappeventlabel,eventdatetime,rowuuid,isfirstgoalactivity,event_rank
0,goal,False,com.kmong.iOS,4.0.4,iPhone,Apple,iOS11.4.1,F36FAA62-ADAC-4AA5-9B00-1FD6CB7EE957,unattributed,unattributed,NaN,NaN,NaN,home.view,NaN,2018-09-28T00:00:00+09:00,fd2a188c-bc9b-4702-9c47-b546b2614817,False,True
1,goal,False,com.kmong.kmong,3.3.5,SM-N935S,samsung,Android7.0,8a871e50-0717-4aed-9bad-04ac3c3793be,unattributed,unattributed,NaN,NaN,NaN,gig_detail.view,41201.0,2018-09-28T00:00:00+09:00,e62dccef-dd70-4415-8a33-c8324ddaed38,False,True
2,goal,False,com.kmong.iOS,4.0.4,iPhone,Apple,iOS12.0,A9E5778A-8F3D-4597-9718-74BF953A9F64,unattributed,unattributed,NaN,NaN,NaN,inbox_detail.view,NaN,2018-09-28T00:00:00+09:00,14eb3197-db83-493a-b7be-83582960c40b,False,True
3,foreground,NaN,com.kmong.iOS,4.0.4,iPhone,Apple,iOS11.4.1,168761CB-CB67-4592-867D-52780D651297,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-09-28T00:00:01+09:00,f9bb91af-248b-44dc-9f5c-1c00b37ea97b,NaN,True
4,goal,False,com.kmong.iOS,4.0.4,iPhone,Apple,iOS11.4.1,ACABB7C0-4C76-413A-B314-E5D6DA0D0E5D,viral,WEB,NaN,NaN,NaN,buyer_order_track.view,NaN,2018-09-28T00:00:02+09:00,236e9946-7801-4898-b609-06c8ab1139dc,False,True


**kmong-funnel.csv**

퍼널(Funnel) 정보를 나타내는 ```kmong-funnel.csv``` . 퍼널은 사용자가 서비스에 접속한 뒤 상품을 구매하기까지의 모든 경로를 구조화 한 것을 의미. 

사용자가 특정 상품을 결제하기 위해선, 해당 상품의 설명을 적은 페이지를 방문해야 하며, 상품 설명 페이지를 방문하기 위해서는 해당 상품이 리스트되어있는 페이지를 방문함. 이런 방식으로 웹사이트를 설명하는 것을 일명 퍼널이라 한다.


이 퍼널 정보를 이용해 사용자가 상품을 구매하는 전환(Conversion) 여부, 내지는 서비스를 이탈하는 이탈(Churn) 여부를 추적할 수 있으며, 이를 통해서 웹사이트나 모바일서비스를 개선하여 전환율을 높이고 이탈률을 낮출 수 있다. 크몽에서는 서비스에서 사용하는 퍼널에 대한 설명을 ```kmong-funnel.csv``` 파일에 저장해두고 있다.

In [4]:

raw_funnel = pd.read_csv("./kmong/data/kmong-funnel.csv")

print(raw_funnel.shape)

raw_funnel

(53, 6)


,Lv2,viewid,viewid desc,Lv1,funnel name,funnel desc
0,1100,home,홈 (탭),11,home,홈
1,1210,category_list,카테고리 목록 (탭),12,category,카테고리
2,1200,category_gig,카테고리-상품목록,12,category,카테고리
3,1300,search,검색,13,search,검색
4,1301,search_gig,검색-상품목록,13,search,검색
5,1302,search_seller,검색-전문가,13,search,검색
6,1400,gig_detail,상품상세,14,gig,상품
7,1401,gig_detail_option,상품상세-상품선택,14,gig,상품
8,1420,profile,전문가프로필,14,gig,상품
9,1520,login_sns,간편로그인,15,login,로그인


**kmong-category.csv**

크몽에서 판매하는 상품의 카테고리를 나타내는 ```kmong-category.csv``` . 크몽에서는 디자인, 번역, 컨텐츠 제작등 다양한 상품을 판매하고 있는데, 이 상품을 그룹화하고 정리할 수 있도록 만든 것이 카테고리다. 크몽에서는 이 값을 ```kmong-category.csv``` 라는 변수에 저장해두고 있다.

In [5]:

raw_category = pd.read_csv("./kmong/data/kmong-category.csv")

print(raw_category.shape)

raw_category

(245, 9)


,depth,categoryid,categoryname,cat1_id,cat2_id,cat3_id,cat1,cat2,cat3
0,1,1,디자인,1,NaN,NaN,디자인,NaN,NaN
1,1,2,마케팅,2,NaN,NaN,마케팅,NaN,NaN
2,1,3,번역·통역,3,NaN,NaN,번역·통역,NaN,NaN
3,1,4,문서작성,4,NaN,NaN,문서작성,NaN,NaN
4,1,6,IT·프로그래밍,6,NaN,NaN,IT·프로그래밍,NaN,NaN
...,...,...,...,...,...,...,...,...,...
240,3,72501,모델,7,725.0,72501.0,콘텐츠 제작,엔터테이너,모델
241,3,72502,MC,7,725.0,72502.0,콘텐츠 제작,엔터테이너,MC
242,3,72503,배우,7,725.0,72503.0,콘텐츠 제작,엔터테이너,배우
243,3,72504,공연,7,725.0,72504.0,콘텐츠 제작,엔터테이너,공연


### 데이터 전처리

In [7]:
raw_log["userid"]=raw_log["canonicaldeviceuuid"]
raw_log[["canonicaldeviceuuid","userid"]]


,canonicaldeviceuuid,userid
0,F36FAA62-ADAC-4AA5-9B00-1FD6CB7EE957,F36FAA62-ADAC-4AA5-9B00-1FD6CB7EE957
1,8a871e50-0717-4aed-9bad-04ac3c3793be,8a871e50-0717-4aed-9bad-04ac3c3793be
2,A9E5778A-8F3D-4597-9718-74BF953A9F64,A9E5778A-8F3D-4597-9718-74BF953A9F64
3,168761CB-CB67-4592-867D-52780D651297,168761CB-CB67-4592-867D-52780D651297
4,ACABB7C0-4C76-413A-B314-E5D6DA0D0E5D,ACABB7C0-4C76-413A-B314-E5D6DA0D0E5D
...,...,...
434239,1fbc9ad2-8109-4294-ae9e-9acf6dce72c3,1fbc9ad2-8109-4294-ae9e-9acf6dce72c3
434240,ea69fbd8-16fe-4db4-ae1e-512d5bbd1d6f,ea69fbd8-16fe-4db4-ae1e-512d5bbd1d6f
434241,1fbc9ad2-8109-4294-ae9e-9acf6dce72c3,1fbc9ad2-8109-4294-ae9e-9acf6dce72c3
434242,ea69fbd8-16fe-4db4-ae1e-512d5bbd1d6f,ea69fbd8-16fe-4db4-ae1e-512d5bbd1d6f


In [8]:
raw_log["eventdatetime"] = pd.to_datetime(raw_log["eventdatetime"])
raw_log["eventdatetime_year"]=raw_log["eventdatetime"].dt.year
raw_log["eventdatetime_month"]=raw_log["eventdatetime"].dt.month
raw_log["eventdatetime_day"]=raw_log["eventdatetime"].dt.day
raw_log["eventdatetime_hour"]=raw_log["eventdatetime"].dt.hour
raw_log["eventdatetime_minute"]=raw_log["eventdatetime"].dt.minute
raw_log["eventdatetime_second"]=raw_log["eventdatetime"].dt.second


In [9]:
raw_log.columns

Index(['eventcategory', 'isfirstactivity', 'apppackagename', 'appversion',
       'devicetype', 'devicemanufacturer', 'osversion', 'canonicaldeviceuuid',
       'sourcetype', 'channel', 'params_campaign', 'params_medium',
       'params_term', 'inappeventcategory', 'inappeventlabel', 'eventdatetime',
       'rowuuid', 'isfirstgoalactivity', 'event_rank', 'userid',
       'eventdatetime_year', 'eventdatetime_month', 'eventdatetime_day',
       'eventdatetime_hour', 'eventdatetime_minute', 'eventdatetime_second'],
      dtype='object')

In [10]:
raw_log[["eventdatetime","eventdatetime_year","eventdatetime_month","eventdatetime_day","eventdatetime_hour","eventdatetime_minute","eventdatetime_second"]]

,eventdatetime,eventdatetime_year,eventdatetime_month,eventdatetime_day,eventdatetime_hour,eventdatetime_minute,eventdatetime_second
0,2018-09-28 00:00:00+09:00,2018,9,28,0,0,0
1,2018-09-28 00:00:00+09:00,2018,9,28,0,0,0
2,2018-09-28 00:00:00+09:00,2018,9,28,0,0,0
3,2018-09-28 00:00:01+09:00,2018,9,28,0,0,1
4,2018-09-28 00:00:02+09:00,2018,9,28,0,0,2
...,...,...,...,...,...,...,...
434239,2018-09-29 23:59:52+09:00,2018,9,29,23,59,52
434240,2018-09-29 23:59:53+09:00,2018,9,29,23,59,53
434241,2018-09-29 23:59:53+09:00,2018,9,29,23,59,53
434242,2018-09-29 23:59:54+09:00,2018,9,29,23,59,54


In [11]:
def clean_ostype(osversion):
    if "iOS" in osversion:
        return "iOS"
    elif "Android" in osversion:
        return "Android"
    
def clean_osversion(osversion):
    return osversion.replace("iOS","").replace("Android","")


In [12]:
clean_ostype("iOS11.4.1")

'iOS'

In [13]:
clean_osversion("iOS11.4.1")

'11.4.1'

In [14]:

raw_log["ostype(clean)"]=raw_log["osversion"].apply(clean_ostype)

raw_log["osversion(clean)"]=raw_log["osversion"].apply(clean_osversion)


In [15]:
raw_log[["osversion","ostype(clean)","osversion(clean)"]]

,osversion,ostype(clean),osversion(clean)
0,iOS11.4.1,iOS,11.4.1
1,Android7.0,Android,7.0
2,iOS12.0,iOS,12.0
3,iOS11.4.1,iOS,11.4.1
4,iOS11.4.1,iOS,11.4.1
...,...,...,...
434239,Android8.0.0,Android,8.0.0
434240,Android8.0.0,Android,8.0.0
434241,Android8.0.0,Android,8.0.0
434242,Android8.0.0,Android,8.0.0


In [17]:
raw_log.head()


,eventcategory,isfirstactivity,apppackagename,appversion,devicetype,devicemanufacturer,osversion,canonicaldeviceuuid,sourcetype,channel,params_campaign,params_medium,params_term,inappeventcategory,inappeventlabel,eventdatetime,rowuuid,isfirstgoalactivity,event_rank,userid,eventdatetime_year,eventdatetime_month,eventdatetime_day,eventdatetime_hour,eventdatetime_minute,eventdatetime_second,ostype(clean),osversion(clean)
0,goal,False,com.kmong.iOS,4.0.4,iPhone,Apple,iOS11.4.1,F36FAA62-ADAC-4AA5-9B00-1FD6CB7EE957,unattributed,unattributed,NaN,NaN,NaN,home.view,NaN,2018-09-28 00:00:00+09:00,fd2a188c-bc9b-4702-9c47-b546b2614817,False,True,F36FAA62-ADAC-4AA5-9B00-1FD6CB7EE957,2018,9,28,0,0,0,iOS,11.4.1
1,goal,False,com.kmong.kmong,3.3.5,SM-N935S,samsung,Android7.0,8a871e50-0717-4aed-9bad-04ac3c3793be,unattributed,unattributed,NaN,NaN,NaN,gig_detail.view,41201.0,2018-09-28 00:00:00+09:00,e62dccef-dd70-4415-8a33-c8324ddaed38,False,True,8a871e50-0717-4aed-9bad-04ac3c3793be,2018,9,28,0,0,0,Android,7.0
2,goal,False,com.kmong.iOS,4.0.4,iPhone,Apple,iOS12.0,A9E5778A-8F3D-4597-9718-74BF953A9F64,unattributed,unattributed,NaN,NaN,NaN,inbox_detail.view,NaN,2018-09-28 00:00:00+09:00,14eb3197-db83-493a-b7be-83582960c40b,False,True,A9E5778A-8F3D-4597-9718-74BF953A9F64,2018,9,28,0,0,0,iOS,12.0
3,foreground,NaN,com.kmong.iOS,4.0.4,iPhone,Apple,iOS11.4.1,168761CB-CB67-4592-867D-52780D651297,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-09-28 00:00:01+09:00,f9bb91af-248b-44dc-9f5c-1c00b37ea97b,NaN,True,168761CB-CB67-4592-867D-52780D651297,2018,9,28,0,0,1,iOS,11.4.1
4,goal,False,com.kmong.iOS,4.0.4,iPhone,Apple,iOS11.4.1,ACABB7C0-4C76-413A-B314-E5D6DA0D0E5D,viral,WEB,NaN,NaN,NaN,buyer_order_track.view,NaN,2018-09-28 00:00:02+09:00,236e9946-7801-4898-b609-06c8ab1139dc,False,True,ACABB7C0-4C76-413A-B314-E5D6DA0D0E5D,2018,9,28,0,0,2,iOS,11.4.1


In [20]:
def clean_manufacturer(menu):
    if menu == "samsung":
        return "Samsung"
    elif menu == "Apple":
        return "Apple"
    elif "LG" in menu:
        return "LG"
    else:
        return "Others"

In [21]:
raw_log["devicemanufacturer(clean)"]=raw_log["devicemanufacturer"].apply(clean_manufacturer)
raw_log["devicemanufacturer(clean)"].value_counts()

Samsung    230313
Apple      163566
LG          32649
Others       7716
Name: devicemanufacturer(clean), dtype: int64

In [22]:
import numpy as np
np.nan==np.nan

False

In [23]:
pd.isnull(np.nan)

True

In [24]:
def clean_channel(channel):
    if pd.isnull(channel):
        return np.nan
    elif "daum" in channel:
        return "daum"
    elif "naver" in channel:
        return "naver"
    elif "apple" in channel:
        return "apple"
    elif "facebook" in channel:
        return "facebook"
    elif "WEB" in channel:
        return "web"
    else:
        return channel

In [25]:
raw_log["channel(clean)"]=raw_log["channel"].apply(clean_channel)

In [26]:
raw_log["channel(clean)"].value_counts()

unattributed      270834
web                27639
google-play        19973
google.adwords      4076
google              3186
(not set)           2444
naver               1224
daum                 966
apple                697
facebook             126
Name: channel(clean), dtype: int64

In [27]:
def clean_viewcategory(category):
    if pd.isnull(category):
        return category
    
    else:
        return category.split(".")[0].split("_")[0]

def clean_viewid(category):
    if pd.isnull(category):
        return category
    elif "." in category:
        return category.split(".")[0]
    
def clean_viewaction(category):
    if pd.isnull(category):
        return category
    elif "." in category:
        return category.split(".")[-1]

In [28]:
clean_viewid("home.view")

'home'

In [29]:
clean_viewaction("home.view")

'view'

In [30]:
raw_log["inappeventcategory"].apply(clean_viewcategory).head(10)

0     home
1      gig
2    inbox
3      NaN
4    buyer
5     home
6      gig
7      NaN
8      NaN
9    buyer
Name: inappeventcategory, dtype: object

In [31]:
raw_log["viewcategory"]=raw_log["inappeventcategory"].apply(clean_viewcategory)
raw_log["viewid"]=raw_log["inappeventcategory"].apply(clean_viewid)
raw_log["viewaction"]=raw_log["inappeventcategory"].apply(clean_viewaction)

In [32]:
raw_log[["viewcategory","viewid","viewaction"]]

,viewcategory,viewid,viewaction
0,home,home,view
1,gig,gig_detail,view
2,inbox,inbox_detail,view
3,NaN,NaN,NaN
4,buyer,buyer_order_track,view
...,...,...,...
434239,gig,gig_detail,view
434240,home,home,view
434241,category,category_gig,view
434242,NaN,NaN,NaN


In [33]:
cols= "viewcategory	viewid	viewaction"
cols=cols.split("\t")
raw_log[cols].head()

,viewcategory,viewid,viewaction
0,home,home,view
1,gig,gig_detail,view
2,inbox,inbox_detail,view
3,NaN,NaN,NaN
4,buyer,buyer_order_track,view


In [34]:
raw_log.head()

,eventcategory,isfirstactivity,apppackagename,appversion,devicetype,devicemanufacturer,osversion,canonicaldeviceuuid,sourcetype,channel,params_campaign,params_medium,params_term,inappeventcategory,inappeventlabel,eventdatetime,rowuuid,isfirstgoalactivity,event_rank,userid,eventdatetime_year,eventdatetime_month,eventdatetime_day,eventdatetime_hour,eventdatetime_minute,eventdatetime_second,ostype(clean),osversion(clean),devicemanufacturer(clean),channel(clean),viewcategory,viewid,viewaction
0,goal,False,com.kmong.iOS,4.0.4,iPhone,Apple,iOS11.4.1,F36FAA62-ADAC-4AA5-9B00-1FD6CB7EE957,unattributed,unattributed,NaN,NaN,NaN,home.view,NaN,2018-09-28 00:00:00+09:00,fd2a188c-bc9b-4702-9c47-b546b2614817,False,True,F36FAA62-ADAC-4AA5-9B00-1FD6CB7EE957,2018,9,28,0,0,0,iOS,11.4.1,Apple,unattributed,home,home,view
1,goal,False,com.kmong.kmong,3.3.5,SM-N935S,samsung,Android7.0,8a871e50-0717-4aed-9bad-04ac3c3793be,unattributed,unattributed,NaN,NaN,NaN,gig_detail.view,41201.0,2018-09-28 00:00:00+09:00,e62dccef-dd70-4415-8a33-c8324ddaed38,False,True,8a871e50-0717-4aed-9bad-04ac3c3793be,2018,9,28,0,0,0,Android,7.0,Samsung,unattributed,gig,gig_detail,view
2,goal,False,com.kmong.iOS,4.0.4,iPhone,Apple,iOS12.0,A9E5778A-8F3D-4597-9718-74BF953A9F64,unattributed,unattributed,NaN,NaN,NaN,inbox_detail.view,NaN,2018-09-28 00:00:00+09:00,14eb3197-db83-493a-b7be-83582960c40b,False,True,A9E5778A-8F3D-4597-9718-74BF953A9F64,2018,9,28,0,0,0,iOS,12.0,Apple,unattributed,inbox,inbox_detail,view
3,foreground,NaN,com.kmong.iOS,4.0.4,iPhone,Apple,iOS11.4.1,168761CB-CB67-4592-867D-52780D651297,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-09-28 00:00:01+09:00,f9bb91af-248b-44dc-9f5c-1c00b37ea97b,NaN,True,168761CB-CB67-4592-867D-52780D651297,2018,9,28,0,0,1,iOS,11.4.1,Apple,NaN,NaN,NaN,NaN
4,goal,False,com.kmong.iOS,4.0.4,iPhone,Apple,iOS11.4.1,ACABB7C0-4C76-413A-B314-E5D6DA0D0E5D,viral,WEB,NaN,NaN,NaN,buyer_order_track.view,NaN,2018-09-28 00:00:02+09:00,236e9946-7801-4898-b609-06c8ab1139dc,False,True,ACABB7C0-4C76-413A-B314-E5D6DA0D0E5D,2018,9,28,0,0,2,iOS,11.4.1,Apple,web,buyer,buyer_order_track,view


In [35]:
cols="eventcategory	isfirstactivity	apppackagename	appversion	devicetype	sourcetype	params_campaign	params_medium	params_term	inappeventcategory	inappeventlabel	eventdatetime	rowuuid	isfirstgoalactivity	ostype(clean)	osversion(clean)	devicemanufacturer(clean)	userid	channel(clean)	eventdatetime_year	eventdatetime_month	eventdatetime_day	eventdatetime_hour	eventdatetime_minute	eventdatetime_second	viewid	viewaction	viewcategory"
cols=cols.split("\t")
log=raw_log[cols].copy()
log.head()

,eventcategory,isfirstactivity,apppackagename,appversion,devicetype,sourcetype,params_campaign,params_medium,params_term,inappeventcategory,inappeventlabel,eventdatetime,rowuuid,isfirstgoalactivity,ostype(clean),osversion(clean),devicemanufacturer(clean),userid,channel(clean),eventdatetime_year,eventdatetime_month,eventdatetime_day,eventdatetime_hour,eventdatetime_minute,eventdatetime_second,viewid,viewaction,viewcategory
0,goal,False,com.kmong.iOS,4.0.4,iPhone,unattributed,NaN,NaN,NaN,home.view,NaN,2018-09-28 00:00:00+09:00,fd2a188c-bc9b-4702-9c47-b546b2614817,False,iOS,11.4.1,Apple,F36FAA62-ADAC-4AA5-9B00-1FD6CB7EE957,unattributed,2018,9,28,0,0,0,home,view,home
1,goal,False,com.kmong.kmong,3.3.5,SM-N935S,unattributed,NaN,NaN,NaN,gig_detail.view,41201.0,2018-09-28 00:00:00+09:00,e62dccef-dd70-4415-8a33-c8324ddaed38,False,Android,7.0,Samsung,8a871e50-0717-4aed-9bad-04ac3c3793be,unattributed,2018,9,28,0,0,0,gig_detail,view,gig
2,goal,False,com.kmong.iOS,4.0.4,iPhone,unattributed,NaN,NaN,NaN,inbox_detail.view,NaN,2018-09-28 00:00:00+09:00,14eb3197-db83-493a-b7be-83582960c40b,False,iOS,12.0,Apple,A9E5778A-8F3D-4597-9718-74BF953A9F64,unattributed,2018,9,28,0,0,0,inbox_detail,view,inbox
3,foreground,NaN,com.kmong.iOS,4.0.4,iPhone,NaN,NaN,NaN,NaN,NaN,NaN,2018-09-28 00:00:01+09:00,f9bb91af-248b-44dc-9f5c-1c00b37ea97b,NaN,iOS,11.4.1,Apple,168761CB-CB67-4592-867D-52780D651297,NaN,2018,9,28,0,0,1,NaN,NaN,NaN
4,goal,False,com.kmong.iOS,4.0.4,iPhone,viral,NaN,NaN,NaN,buyer_order_track.view,NaN,2018-09-28 00:00:02+09:00,236e9946-7801-4898-b609-06c8ab1139dc,False,iOS,11.4.1,Apple,ACABB7C0-4C76-413A-B314-E5D6DA0D0E5D,web,2018,9,28,0,0,2,buyer_order_track,view,buyer


<table border="1" class="dataframe" style="float: left;">
    <thead>
        <tr style="text-align: right;">
            <th></th>
            <th>eventcategory</th>
            <th>isfirstactivity</th>
            <th>apppackagename</th>
            <th>appversion</th>
            <th>devicetype</th>
            <th>sourcetype</th>
            <th>params_campaign</th>
            <th>params_medium</th>
            <th>params_term</th>
            <th>inappeventcategory</th>
            <th>inappeventlabel</th>
            <th>eventdatetime</th>
            <th>rowuuid</th>
            <th>isfirstgoalactivity</th>
            <th>ostype(clean)</th>
            <th>osversion(clean)</th>
            <th>devicemanufacturer(clean)</th>
            <th>userid</th>
            <th>channel(clean)</th>
            <th>eventdatetime_year</th>
            <th>eventdatetime_month</th>
            <th>eventdatetime_day</th>
            <th>eventdatetime_hour</th>
            <th>eventdatetime_minute</th>
            <th>eventdatetime_second</th>
            <th>viewid</th>
            <th>viewaction</th>
            <th>viewcategory</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <th>0</th>
            <td>goal</td>
            <td>False</td>
            <td>com.kmong.iOS</td>
            <td>4.0.4</td>
            <td>iPhone</td>
            <td>unattributed</td>
            <td>NaN</td>
            <td>NaN</td>
            <td>NaN</td>
            <td>home.view</td>
            <td>NaN</td>
            <td>2018-09-27 15:00:00</td>
            <td>fd2a188c-bc9b-4702-9c47-b546b2614817</td>
            <td>False</td>
            <td>iOS</td>
            <td>11.4.1</td>
            <td>Apple</td>
            <td>F36FAA62-ADAC-4AA5-9B00-1FD6CB7EE957</td>
            <td>unattributed</td>
            <td>2018</td>
            <td>9</td>
            <td>27</td>
            <td>15</td>
            <td>0</td>
            <td>0</td>
            <td>home</td>
            <td>view</td>
            <td>home</td>
        </tr>
        <tr>
            <th>1</th>
            <td>goal</td>
            <td>False</td>
            <td>com.kmong.kmong</td>
            <td>3.3.5</td>
            <td>SM-N935S</td>
            <td>unattributed</td>
            <td>NaN</td>
            <td>NaN</td>
            <td>NaN</td>
            <td>gig_detail.view</td>
            <td>41201.0</td>
            <td>2018-09-27 15:00:00</td>
            <td>e62dccef-dd70-4415-8a33-c8324ddaed38</td>
            <td>False</td>
            <td>Android</td>
            <td>7.0</td>
            <td>Samsung</td>
            <td>8a871e50-0717-4aed-9bad-04ac3c3793be</td>
            <td>unattributed</td>
            <td>2018</td>
            <td>9</td>
            <td>27</td>
            <td>15</td>
            <td>0</td>
            <td>0</td>
            <td>gig_detail</td>
            <td>view</td>
            <td>gig</td>
        </tr>
        <tr>
            <th>2</th>
            <td>goal</td>
            <td>False</td>
            <td>com.kmong.iOS</td>
            <td>4.0.4</td>
            <td>iPhone</td>
            <td>unattributed</td>
            <td>NaN</td>
            <td>NaN</td>
            <td>NaN</td>
            <td>inbox_detail.view</td>
            <td>NaN</td>
            <td>2018-09-27 15:00:00</td>
            <td>14eb3197-db83-493a-b7be-83582960c40b</td>
            <td>False</td>
            <td>iOS</td>
            <td>12.0</td>
            <td>Apple</td>
            <td>A9E5778A-8F3D-4597-9718-74BF953A9F64</td>
            <td>unattributed</td>
            <td>2018</td>
            <td>9</td>
            <td>27</td>
            <td>15</td>
            <td>0</td>
            <td>0</td>
            <td>inbox_detail</td>
            <td>view</td>
            <td>inbox</td>
        </tr>
        <tr>
            <th>3</th>
            <td>foreground</td>
            <td>NaN</td>
            <td>com.kmong.iOS</td>
            <td>4.0.4</td>
            <td>iPhone</td>
            <td>NaN</td>
            <td>NaN</td>
            <td>NaN</td>
            <td>NaN</td>
            <td>NaN</td>
            <td>NaN</td>
            <td>2018-09-27 15:00:01</td>
            <td>f9bb91af-248b-44dc-9f5c-1c00b37ea97b</td>
            <td>NaN</td>
            <td>iOS</td>
            <td>11.4.1</td>
            <td>Apple</td>
            <td>168761CB-CB67-4592-867D-52780D651297</td>
            <td>NaN</td>
            <td>2018</td>
            <td>9</td>
            <td>27</td>
            <td>15</td>
            <td>0</td>
            <td>1</td>
            <td>NaN</td>
            <td>NaN</td>
            <td>NaN</td>
        </tr>
        <tr>
            <th>4</th>
            <td>goal</td>
            <td>False</td>
            <td>com.kmong.iOS</td>
            <td>4.0.4</td>
            <td>iPhone</td>
            <td>viral</td>
            <td>NaN</td>
            <td>NaN</td>
            <td>NaN</td>
            <td>buyer_order_track.view</td>
            <td>NaN</td>
            <td>2018-09-27 15:00:02</td>
            <td>236e9946-7801-4898-b609-06c8ab1139dc</td>
            <td>False</td>
            <td>iOS</td>
            <td>11.4.1</td>
            <td>Apple</td>
            <td>ACABB7C0-4C76-413A-B314-E5D6DA0D0E5D</td>
            <td>web</td>
            <td>2018</td>
            <td>9</td>
            <td>27</td>
            <td>15</td>
            <td>0</td>
            <td>2</td>
            <td>buyer_order_track</td>
            <td>view</td>
            <td>buyer</td>
        </tr>
    </tbody>
</table>

In [36]:
new_cols="event_category	is_first_activity	app_package_name	app_version	device_type	source_type	params_campaign	params_medium	params_term	in_app_event_category	in_app_event_label	event_datetime	row_uuid	is_first_goal_activity	os_type	os_version	device_manufacturer	user_id	channel	event_datetime_year	event_datetime_month	event_datetime_day	event_datetime_hour	event_datetime_minute	event_datetime_second	view_id	view_action	view_category"
new_cols=new_cols.split("\t")
new_cols[:5]

['event_category',
 'is_first_activity',
 'app_package_name',
 'app_version',
 'device_type']

In [37]:
log.columns=new_cols
log.head()

,event_category,is_first_activity,app_package_name,app_version,device_type,source_type,params_campaign,params_medium,params_term,in_app_event_category,in_app_event_label,event_datetime,row_uuid,is_first_goal_activity,os_type,os_version,device_manufacturer,user_id,channel,event_datetime_year,event_datetime_month,event_datetime_day,event_datetime_hour,event_datetime_minute,event_datetime_second,view_id,view_action,view_category
0,goal,False,com.kmong.iOS,4.0.4,iPhone,unattributed,NaN,NaN,NaN,home.view,NaN,2018-09-28 00:00:00+09:00,fd2a188c-bc9b-4702-9c47-b546b2614817,False,iOS,11.4.1,Apple,F36FAA62-ADAC-4AA5-9B00-1FD6CB7EE957,unattributed,2018,9,28,0,0,0,home,view,home
1,goal,False,com.kmong.kmong,3.3.5,SM-N935S,unattributed,NaN,NaN,NaN,gig_detail.view,41201.0,2018-09-28 00:00:00+09:00,e62dccef-dd70-4415-8a33-c8324ddaed38,False,Android,7.0,Samsung,8a871e50-0717-4aed-9bad-04ac3c3793be,unattributed,2018,9,28,0,0,0,gig_detail,view,gig
2,goal,False,com.kmong.iOS,4.0.4,iPhone,unattributed,NaN,NaN,NaN,inbox_detail.view,NaN,2018-09-28 00:00:00+09:00,14eb3197-db83-493a-b7be-83582960c40b,False,iOS,12.0,Apple,A9E5778A-8F3D-4597-9718-74BF953A9F64,unattributed,2018,9,28,0,0,0,inbox_detail,view,inbox
3,foreground,NaN,com.kmong.iOS,4.0.4,iPhone,NaN,NaN,NaN,NaN,NaN,NaN,2018-09-28 00:00:01+09:00,f9bb91af-248b-44dc-9f5c-1c00b37ea97b,NaN,iOS,11.4.1,Apple,168761CB-CB67-4592-867D-52780D651297,NaN,2018,9,28,0,0,1,NaN,NaN,NaN
4,goal,False,com.kmong.iOS,4.0.4,iPhone,viral,NaN,NaN,NaN,buyer_order_track.view,NaN,2018-09-28 00:00:02+09:00,236e9946-7801-4898-b609-06c8ab1139dc,False,iOS,11.4.1,Apple,ACABB7C0-4C76-413A-B314-E5D6DA0D0E5D,web,2018,9,28,0,0,2,buyer_order_track,view,buyer


In [38]:
raw_log=log
raw_log

,event_category,is_first_activity,app_package_name,app_version,device_type,source_type,params_campaign,params_medium,params_term,in_app_event_category,in_app_event_label,event_datetime,row_uuid,is_first_goal_activity,os_type,os_version,device_manufacturer,user_id,channel,event_datetime_year,event_datetime_month,event_datetime_day,event_datetime_hour,event_datetime_minute,event_datetime_second,view_id,view_action,view_category
0,goal,False,com.kmong.iOS,4.0.4,iPhone,unattributed,NaN,NaN,NaN,home.view,NaN,2018-09-28 00:00:00+09:00,fd2a188c-bc9b-4702-9c47-b546b2614817,False,iOS,11.4.1,Apple,F36FAA62-ADAC-4AA5-9B00-1FD6CB7EE957,unattributed,2018,9,28,0,0,0,home,view,home
1,goal,False,com.kmong.kmong,3.3.5,SM-N935S,unattributed,NaN,NaN,NaN,gig_detail.view,41201.0,2018-09-28 00:00:00+09:00,e62dccef-dd70-4415-8a33-c8324ddaed38,False,Android,7.0,Samsung,8a871e50-0717-4aed-9bad-04ac3c3793be,unattributed,2018,9,28,0,0,0,gig_detail,view,gig
2,goal,False,com.kmong.iOS,4.0.4,iPhone,unattributed,NaN,NaN,NaN,inbox_detail.view,NaN,2018-09-28 00:00:00+09:00,14eb3197-db83-493a-b7be-83582960c40b,False,iOS,12.0,Apple,A9E5778A-8F3D-4597-9718-74BF953A9F64,unattributed,2018,9,28,0,0,0,inbox_detail,view,inbox
3,foreground,NaN,com.kmong.iOS,4.0.4,iPhone,NaN,NaN,NaN,NaN,NaN,NaN,2018-09-28 00:00:01+09:00,f9bb91af-248b-44dc-9f5c-1c00b37ea97b,NaN,iOS,11.4.1,Apple,168761CB-CB67-4592-867D-52780D651297,NaN,2018,9,28,0,0,1,NaN,NaN,NaN
4,goal,False,com.kmong.iOS,4.0.4,iPhone,viral,NaN,NaN,NaN,buyer_order_track.view,NaN,2018-09-28 00:00:02+09:00,236e9946-7801-4898-b609-06c8ab1139dc,False,iOS,11.4.1,Apple,ACABB7C0-4C76-413A-B314-E5D6DA0D0E5D,web,2018,9,28,0,0,2,buyer_order_track,view,buyer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
434239,goal,False,com.kmong.kmong,3.3.5,SM-G930L,stranger,mweb-app-install,cpc,NaN,gig_detail.view,104.0,2018-09-29 23:59:52+09:00,9d9fd3d8-30c5-49e5-8480-41b536eb4edb,False,Android,8.0.0,Samsung,1fbc9ad2-8109-4294-ae9e-9acf6dce72c3,google,2018,9,29,23,59,52,gig_detail,view,gig
434240,goal,False,com.kmong.kmong,3.3.5,SM-N950N,unattributed,NaN,NaN,NaN,home.view,NaN,2018-09-29 23:59:53+09:00,88653965-4aff-4d2e-8dd6-88f25ac82595,False,Android,8.0.0,Samsung,ea69fbd8-16fe-4db4-ae1e-512d5bbd1d6f,unattributed,2018,9,29,23,59,53,home,view,home
434241,goal,False,com.kmong.kmong,3.3.5,SM-G930L,stranger,mweb-app-install,cpc,NaN,category_gig.view,104.0,2018-09-29 23:59:53+09:00,40bdc87e-265a-4cc2-a1d4-f1da96a0b9dc,False,Android,8.0.0,Samsung,1fbc9ad2-8109-4294-ae9e-9acf6dce72c3,google,2018,9,29,23,59,53,category_gig,view,category
434242,exit,NaN,com.kmong.kmong,3.3.5,SM-N950N,NaN,NaN,NaN,NaN,NaN,NaN,2018-09-29 23:59:54+09:00,f44a488b-1a8b-4f53-ad8e-8b21c695925b,NaN,Android,8.0.0,Samsung,ea69fbd8-16fe-4db4-ae1e-512d5bbd1d6f,NaN,2018,9,29,23,59,54,NaN,NaN,NaN


<table border="1" class="dataframe" style="float: left;">
    <thead>
        <tr style="text-align: right;">
            <th></th>
            <th>event_category</th>
            <th>is_first_activity</th>
            <th>app_package_name</th>
            <th>app_version</th>
            <th>device_type</th>
            <th>source_type</th>
            <th>params_campaign</th>
            <th>params_medium</th>
            <th>params_term</th>
            <th>in_app_event_category</th>
            <th>in_app_event_label</th>
            <th>event_datetime</th>
            <th>row_uuid</th>
            <th>is_first_goal_activity</th>
            <th>os_type</th>
            <th>os_version</th>
            <th>device_manufacturer</th>
            <th>user_id</th>
            <th>channel</th>
            <th>event_datetime_year</th>
            <th>event_datetime_month</th>
            <th>event_datetime_day</th>
            <th>event_datetime_hour</th>
            <th>event_datetime_minute</th>
            <th>event_datetime_second</th>
            <th>view_id</th>
            <th>view_action</th>
            <th>view_category</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <th>0</th>
            <td>goal</td>
            <td>False</td>
            <td>com.kmong.iOS</td>
            <td>4.0.4</td>
            <td>iPhone</td>
            <td>unattributed</td>
            <td>NaN</td>
            <td>NaN</td>
            <td>NaN</td>
            <td>home.view</td>
            <td>NaN</td>
            <td>2018-09-27 15:00:00</td>
            <td>fd2a188c-bc9b-4702-9c47-b546b2614817</td>
            <td>False</td>
            <td>iOS</td>
            <td>11.4.1</td>
            <td>Apple</td>
            <td>F36FAA62-ADAC-4AA5-9B00-1FD6CB7EE957</td>
            <td>unattributed</td>
            <td>2018</td>
            <td>9</td>
            <td>27</td>
            <td>15</td>
            <td>0</td>
            <td>0</td>
            <td>home</td>
            <td>view</td>
            <td>home</td>
        </tr>
        <tr>
            <th>1</th>
            <td>goal</td>
            <td>False</td>
            <td>com.kmong.kmong</td>
            <td>3.3.5</td>
            <td>SM-N935S</td>
            <td>unattributed</td>
            <td>NaN</td>
            <td>NaN</td>
            <td>NaN</td>
            <td>gig_detail.view</td>
            <td>41201.0</td>
            <td>2018-09-27 15:00:00</td>
            <td>e62dccef-dd70-4415-8a33-c8324ddaed38</td>
            <td>False</td>
            <td>Android</td>
            <td>7.0</td>
            <td>Samsung</td>
            <td>8a871e50-0717-4aed-9bad-04ac3c3793be</td>
            <td>unattributed</td>
            <td>2018</td>
            <td>9</td>
            <td>27</td>
            <td>15</td>
            <td>0</td>
            <td>0</td>
            <td>gig_detail</td>
            <td>view</td>
            <td>gig</td>
        </tr>
        <tr>
            <th>2</th>
            <td>goal</td>
            <td>False</td>
            <td>com.kmong.iOS</td>
            <td>4.0.4</td>
            <td>iPhone</td>
            <td>unattributed</td>
            <td>NaN</td>
            <td>NaN</td>
            <td>NaN</td>
            <td>inbox_detail.view</td>
            <td>NaN</td>
            <td>2018-09-27 15:00:00</td>
            <td>14eb3197-db83-493a-b7be-83582960c40b</td>
            <td>False</td>
            <td>iOS</td>
            <td>12.0</td>
            <td>Apple</td>
            <td>A9E5778A-8F3D-4597-9718-74BF953A9F64</td>
            <td>unattributed</td>
            <td>2018</td>
            <td>9</td>
            <td>27</td>
            <td>15</td>
            <td>0</td>
            <td>0</td>
            <td>inbox_detail</td>
            <td>view</td>
            <td>inbox</td>
        </tr>
        <tr>
            <th>3</th>
            <td>foreground</td>
            <td>NaN</td>
            <td>com.kmong.iOS</td>
            <td>4.0.4</td>
            <td>iPhone</td>
            <td>NaN</td>
            <td>NaN</td>
            <td>NaN</td>
            <td>NaN</td>
            <td>NaN</td>
            <td>NaN</td>
            <td>2018-09-27 15:00:01</td>
            <td>f9bb91af-248b-44dc-9f5c-1c00b37ea97b</td>
            <td>NaN</td>
            <td>iOS</td>
            <td>11.4.1</td>
            <td>Apple</td>
            <td>168761CB-CB67-4592-867D-52780D651297</td>
            <td>NaN</td>
            <td>2018</td>
            <td>9</td>
            <td>27</td>
            <td>15</td>
            <td>0</td>
            <td>1</td>
            <td>NaN</td>
            <td>NaN</td>
            <td>NaN</td>
        </tr>
        <tr>
            <th>4</th>
            <td>goal</td>
            <td>False</td>
            <td>com.kmong.iOS</td>
            <td>4.0.4</td>
            <td>iPhone</td>
            <td>viral</td>
            <td>NaN</td>
            <td>NaN</td>
            <td>NaN</td>
            <td>buyer_order_track.view</td>
            <td>NaN</td>
            <td>2018-09-27 15:00:02</td>
            <td>236e9946-7801-4898-b609-06c8ab1139dc</td>
            <td>False</td>
            <td>iOS</td>
            <td>11.4.1</td>
            <td>Apple</td>
            <td>ACABB7C0-4C76-413A-B314-E5D6DA0D0E5D</td>
            <td>web</td>
            <td>2018</td>
            <td>9</td>
            <td>27</td>
            <td>15</td>
            <td>0</td>
            <td>2</td>
            <td>buyer_order_track</td>
            <td>view</td>
            <td>buyer</td>
        </tr>
    </tbody>
</table>

In [39]:
cols="row_uuid	app_package_name	user_id	event_datetime	event_datetime_year	event_datetime_month	event_datetime_day	event_datetime_hour	event_datetime_minute	event_datetime_second	device_manufacturer	device_type	os_type	os_version	app_version	event_category	view_category	view_id	view_action	in_app_event_category	in_app_event_label	source_type	channel	params_campaign	params_medium	params_term	is_first_activity	is_first_goal_activity"
cols=cols.split("\t")
log=raw_log[cols].copy()
log.head()

,row_uuid,app_package_name,user_id,event_datetime,event_datetime_year,event_datetime_month,event_datetime_day,event_datetime_hour,event_datetime_minute,event_datetime_second,device_manufacturer,device_type,os_type,os_version,app_version,event_category,view_category,view_id,view_action,in_app_event_category,in_app_event_label,source_type,channel,params_campaign,params_medium,params_term,is_first_activity,is_first_goal_activity
0,fd2a188c-bc9b-4702-9c47-b546b2614817,com.kmong.iOS,F36FAA62-ADAC-4AA5-9B00-1FD6CB7EE957,2018-09-28 00:00:00+09:00,2018,9,28,0,0,0,Apple,iPhone,iOS,11.4.1,4.0.4,goal,home,home,view,home.view,NaN,unattributed,unattributed,NaN,NaN,NaN,False,False
1,e62dccef-dd70-4415-8a33-c8324ddaed38,com.kmong.kmong,8a871e50-0717-4aed-9bad-04ac3c3793be,2018-09-28 00:00:00+09:00,2018,9,28,0,0,0,Samsung,SM-N935S,Android,7.0,3.3.5,goal,gig,gig_detail,view,gig_detail.view,41201.0,unattributed,unattributed,NaN,NaN,NaN,False,False
2,14eb3197-db83-493a-b7be-83582960c40b,com.kmong.iOS,A9E5778A-8F3D-4597-9718-74BF953A9F64,2018-09-28 00:00:00+09:00,2018,9,28,0,0,0,Apple,iPhone,iOS,12.0,4.0.4,goal,inbox,inbox_detail,view,inbox_detail.view,NaN,unattributed,unattributed,NaN,NaN,NaN,False,False
3,f9bb91af-248b-44dc-9f5c-1c00b37ea97b,com.kmong.iOS,168761CB-CB67-4592-867D-52780D651297,2018-09-28 00:00:01+09:00,2018,9,28,0,0,1,Apple,iPhone,iOS,11.4.1,4.0.4,foreground,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,236e9946-7801-4898-b609-06c8ab1139dc,com.kmong.iOS,ACABB7C0-4C76-413A-B314-E5D6DA0D0E5D,2018-09-28 00:00:02+09:00,2018,9,28,0,0,2,Apple,iPhone,iOS,11.4.1,4.0.4,goal,buyer,buyer_order_track,view,buyer_order_track.view,NaN,viral,web,NaN,NaN,NaN,False,False


In [40]:
raw_funnel.head()

,Lv2,viewid,viewid desc,Lv1,funnel name,funnel desc
0,1100,home,홈 (탭),11,home,홈
1,1210,category_list,카테고리 목록 (탭),12,category,카테고리
2,1200,category_gig,카테고리-상품목록,12,category,카테고리
3,1300,search,검색,13,search,검색
4,1301,search_gig,검색-상품목록,13,search,검색


In [41]:
cols="Lv1	Lv2	viewid	viewid desc	funnel name	funnel desc"
cols=cols.split("\t")
funnel=raw_funnel[cols].copy()
funnel.head()

,Lv1,Lv2,viewid,viewid desc,funnel name,funnel desc
0,11,1100,home,홈 (탭),home,홈
1,12,1210,category_list,카테고리 목록 (탭),category,카테고리
2,12,1200,category_gig,카테고리-상품목록,category,카테고리
3,13,1300,search,검색,search,검색
4,13,1301,search_gig,검색-상품목록,search,검색


In [42]:
new_cols="lv1	lv2	view_id	view_desc	funnel_name	funnel_desc"
new_cols=new_cols.split("\t")
funnel.columns=new_cols
funnel.head()

,lv1,lv2,view_id,view_desc,funnel_name,funnel_desc
0,11,1100,home,홈 (탭),home,홈
1,12,1210,category_list,카테고리 목록 (탭),category,카테고리
2,12,1200,category_gig,카테고리-상품목록,category,카테고리
3,13,1300,search,검색,search,검색
4,13,1301,search_gig,검색-상품목록,search,검색


In [43]:
raw_funnel=funnel

In [44]:
raw_funnel.head()

,lv1,lv2,view_id,view_desc,funnel_name,funnel_desc
0,11,1100,home,홈 (탭),home,홈
1,12,1210,category_list,카테고리 목록 (탭),category,카테고리
2,12,1200,category_gig,카테고리-상품목록,category,카테고리
3,13,1300,search,검색,search,검색
4,13,1301,search_gig,검색-상품목록,search,검색


In [47]:
raw_category.head()

,depth,categoryid,categoryname,cat1_id,cat2_id,cat3_id,cat1,cat2,cat3
0,1,1,디자인,1,NaN,NaN,디자인,NaN,NaN
1,1,2,마케팅,2,NaN,NaN,마케팅,NaN,NaN
2,1,3,번역·통역,3,NaN,NaN,번역·통역,NaN,NaN
3,1,4,문서작성,4,NaN,NaN,문서작성,NaN,NaN
4,1,6,IT·프로그래밍,6,NaN,NaN,IT·프로그래밍,NaN,NaN


In [48]:
cols="depth	categoryid	categoryname	cat1_id	cat2_id	cat3_id	cat1	cat2	cat3"
cols=cols.split("\t")
category=raw_category[cols].copy()
category.head()

,depth,categoryid,categoryname,cat1_id,cat2_id,cat3_id,cat1,cat2,cat3
0,1,1,디자인,1,NaN,NaN,디자인,NaN,NaN
1,1,2,마케팅,2,NaN,NaN,마케팅,NaN,NaN
2,1,3,번역·통역,3,NaN,NaN,번역·통역,NaN,NaN
3,1,4,문서작성,4,NaN,NaN,문서작성,NaN,NaN
4,1,6,IT·프로그래밍,6,NaN,NaN,IT·프로그래밍,NaN,NaN


In [49]:
# Write your code here!
new_cols="depth	category_id	category_name	category1_id	category2_id	category3_id	category1	category2	category3"
new_cols=new_cols.split("\t")
category.columns=new_cols
category.head()

,depth,category_id,category_name,category1_id,category2_id,category3_id,category1,category2,category3
0,1,1,디자인,1,NaN,NaN,디자인,NaN,NaN
1,1,2,마케팅,2,NaN,NaN,마케팅,NaN,NaN
2,1,3,번역·통역,3,NaN,NaN,번역·통역,NaN,NaN
3,1,4,문서작성,4,NaN,NaN,문서작성,NaN,NaN
4,1,6,IT·프로그래밍,6,NaN,NaN,IT·프로그래밍,NaN,NaN


In [50]:
log.head(1)

,row_uuid,app_package_name,user_id,event_datetime,event_datetime_year,event_datetime_month,event_datetime_day,event_datetime_hour,event_datetime_minute,event_datetime_second,device_manufacturer,device_type,os_type,os_version,app_version,event_category,view_category,view_id,view_action,in_app_event_category,in_app_event_label,source_type,channel,params_campaign,params_medium,params_term,is_first_activity,is_first_goal_activity
0,fd2a188c-bc9b-4702-9c47-b546b2614817,com.kmong.iOS,F36FAA62-ADAC-4AA5-9B00-1FD6CB7EE957,2018-09-28 00:00:00+09:00,2018,9,28,0,0,0,Apple,iPhone,iOS,11.4.1,4.0.4,goal,home,home,view,home.view,NaN,unattributed,unattributed,NaN,NaN,NaN,False,False


In [51]:
funnel.head(1)

,lv1,lv2,view_id,view_desc,funnel_name,funnel_desc
0,11,1100,home,홈 (탭),home,홈


In [52]:
category.head(1)

,depth,category_id,category_name,category1_id,category2_id,category3_id,category1,category2,category3
0,1,1,디자인,1,NaN,NaN,디자인,NaN,NaN


In [53]:
funnel.head()

,lv1,lv2,view_id,view_desc,funnel_name,funnel_desc
0,11,1100,home,홈 (탭),home,홈
1,12,1210,category_list,카테고리 목록 (탭),category,카테고리
2,12,1200,category_gig,카테고리-상품목록,category,카테고리
3,13,1300,search,검색,search,검색
4,13,1301,search_gig,검색-상품목록,search,검색


In [54]:
log.head()

,row_uuid,app_package_name,user_id,event_datetime,event_datetime_year,event_datetime_month,event_datetime_day,event_datetime_hour,event_datetime_minute,event_datetime_second,device_manufacturer,device_type,os_type,os_version,app_version,event_category,view_category,view_id,view_action,in_app_event_category,in_app_event_label,source_type,channel,params_campaign,params_medium,params_term,is_first_activity,is_first_goal_activity
0,fd2a188c-bc9b-4702-9c47-b546b2614817,com.kmong.iOS,F36FAA62-ADAC-4AA5-9B00-1FD6CB7EE957,2018-09-28 00:00:00+09:00,2018,9,28,0,0,0,Apple,iPhone,iOS,11.4.1,4.0.4,goal,home,home,view,home.view,NaN,unattributed,unattributed,NaN,NaN,NaN,False,False
1,e62dccef-dd70-4415-8a33-c8324ddaed38,com.kmong.kmong,8a871e50-0717-4aed-9bad-04ac3c3793be,2018-09-28 00:00:00+09:00,2018,9,28,0,0,0,Samsung,SM-N935S,Android,7.0,3.3.5,goal,gig,gig_detail,view,gig_detail.view,41201.0,unattributed,unattributed,NaN,NaN,NaN,False,False
2,14eb3197-db83-493a-b7be-83582960c40b,com.kmong.iOS,A9E5778A-8F3D-4597-9718-74BF953A9F64,2018-09-28 00:00:00+09:00,2018,9,28,0,0,0,Apple,iPhone,iOS,12.0,4.0.4,goal,inbox,inbox_detail,view,inbox_detail.view,NaN,unattributed,unattributed,NaN,NaN,NaN,False,False
3,f9bb91af-248b-44dc-9f5c-1c00b37ea97b,com.kmong.iOS,168761CB-CB67-4592-867D-52780D651297,2018-09-28 00:00:01+09:00,2018,9,28,0,0,1,Apple,iPhone,iOS,11.4.1,4.0.4,foreground,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,236e9946-7801-4898-b609-06c8ab1139dc,com.kmong.iOS,ACABB7C0-4C76-413A-B314-E5D6DA0D0E5D,2018-09-28 00:00:02+09:00,2018,9,28,0,0,2,Apple,iPhone,iOS,11.4.1,4.0.4,goal,buyer,buyer_order_track,view,buyer_order_track.view,NaN,viral,web,NaN,NaN,NaN,False,False


In [55]:
df1=pd.merge(log,funnel,on='view_id')

In [56]:
df1

,row_uuid,app_package_name,user_id,event_datetime,event_datetime_year,event_datetime_month,event_datetime_day,event_datetime_hour,event_datetime_minute,event_datetime_second,device_manufacturer,device_type,os_type,os_version,app_version,event_category,view_category,view_id,view_action,in_app_event_category,in_app_event_label,source_type,channel,params_campaign,params_medium,params_term,is_first_activity,is_first_goal_activity,lv1,lv2,view_desc,funnel_name,funnel_desc
0,fd2a188c-bc9b-4702-9c47-b546b2614817,com.kmong.iOS,F36FAA62-ADAC-4AA5-9B00-1FD6CB7EE957,2018-09-28 00:00:00+09:00,2018,9,28,0,0,0,Apple,iPhone,iOS,11.4.1,4.0.4,goal,home,home,view,home.view,NaN,unattributed,unattributed,NaN,NaN,NaN,False,False,11,1100,홈 (탭),home,홈
1,db8ce65f-3707-4c1b-8361-51bcb9d4ecd9,com.kmong.kmong,6d1ea29b-5d5d-4c33-b2cf-7c68b41ce638,2018-09-28 00:00:02+09:00,2018,9,28,0,0,2,Samsung,SM-G955N,Android,8.0.0,3.3.5,goal,home,home,view,home.view,NaN,unattributed,unattributed,NaN,NaN,NaN,False,False,11,1100,홈 (탭),home,홈
2,ff3f2e83-13ea-46af-a500-73811ed4ecc9,com.kmong.kmong,0e5dec96-3f73-4542-b6b6-2031fb2171ad,2018-09-28 00:00:03+09:00,2018,9,28,0,0,3,Samsung,SM-A530N,Android,7.1.1,3.3.3,goal,home,home,view,home.view,NaN,unattributed,unattributed,NaN,NaN,NaN,False,False,11,1100,홈 (탭),home,홈
3,91742336-3566-4be9-bee7-6e04cee7cae7,com.kmong.kmong,a3b89a23-034a-4f4c-8414-adda3dc711cd,2018-09-28 00:00:10+09:00,2018,9,28,0,0,10,Samsung,SM-G930S,Android,8.0.0,3.3.5,goal,home,home,view,home.view,NaN,unattributed,unattributed,NaN,NaN,NaN,False,False,11,1100,홈 (탭),home,홈
4,8e75d85f-4b94-4301-9cee-ba14bebd1280,com.kmong.kmong,1a2f843c-759b-4bab-a368-751d76f2c4e8,2018-09-28 00:00:11+09:00,2018,9,28,0,0,11,LG,LG-F800S,Android,8.0.0,3.3.5,goal,home,home,view,home.view,NaN,app-market,google-play,NaN,organic,NaN,False,False,11,1100,홈 (탭),home,홈
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304545,b7ecede1-c469-48d0-8b4a-6fb8a9ce7760,com.kmong.iOS,06C32753-7678-4E34-AC3F-625FA7B5D115,2018-09-29 15:19:19+09:00,2018,9,29,15,19,19,Apple,iPhone,iOS,11.4.1,4.0.4,goal,kmongCash,kmongCash_history_refundRequest,click,kmongCash_history_refundRequest.click,NaN,unattributed,unattributed,NaN,NaN,NaN,False,False,24,2452,메뉴목록-크몽캐시-캐시환불,transaction_history,거래관리
304546,b888db54-a880-4a43-ad39-257ba533b196,com.kmong.iOS,06C32753-7678-4E34-AC3F-625FA7B5D115,2018-09-29 15:20:05+09:00,2018,9,29,15,20,5,Apple,iPhone,iOS,11.4.1,4.0.4,goal,kmongCash,kmongCash_history_refundRequest,click,kmongCash_history_refundRequest.click,NaN,unattributed,unattributed,NaN,NaN,NaN,False,False,24,2452,메뉴목록-크몽캐시-캐시환불,transaction_history,거래관리
304547,9baeed3c-9026-4b9f-aa0c-0b34d41c0164,com.kmong.kmong,3adb3f17-5982-42ab-9fc9-12681ab1e75a,2018-09-29 15:37:18+09:00,2018,9,29,15,37,18,Samsung,SM-N950N,Android,8.0.0,3.3.2,goal,kmongCash,kmongCash_history_refundRequest,click,kmongCash_history_refundRequest.click,NaN,unattributed,unattributed,NaN,NaN,NaN,False,False,24,2452,메뉴목록-크몽캐시-캐시환불,transaction_history,거래관리
304548,597dc85e-9b87-4afb-a275-716fb3daa6cc,com.kmong.iOS,BC7DE6C9-03E6-44C7-B5AA-68799BB9786B,2018-09-29 20:42:14+09:00,2018,9,29,20,42,14,Apple,iPhone,iOS,11.4.1,4.0.4,goal,kmongCash,kmongCash_history_refundRequest,click,kmongCash_history_refundRequest.click,NaN,viral,web,NaN,NaN,NaN,True,True,24,2452,메뉴목록-크몽캐시-캐시환불,transaction_history,거래관리


In [57]:
df2=pd.merge(df1,category,left_on='in_app_event_label',right_on='category_id')

In [58]:
df2.head()

,row_uuid,app_package_name,user_id,event_datetime,event_datetime_year,event_datetime_month,event_datetime_day,event_datetime_hour,event_datetime_minute,event_datetime_second,device_manufacturer,device_type,os_type,os_version,app_version,event_category,view_category,view_id,view_action,in_app_event_category,in_app_event_label,source_type,channel,params_campaign,params_medium,params_term,is_first_activity,is_first_goal_activity,lv1,lv2,view_desc,funnel_name,funnel_desc,depth,category_id,category_name,category1_id,category2_id,category3_id,category1,category2,category3
0,e62dccef-dd70-4415-8a33-c8324ddaed38,com.kmong.kmong,8a871e50-0717-4aed-9bad-04ac3c3793be,2018-09-28 00:00:00+09:00,2018,9,28,0,0,0,Samsung,SM-N935S,Android,7.0,3.3.5,goal,gig,gig_detail,view,gig_detail.view,41201.0,unattributed,unattributed,NaN,NaN,NaN,False,False,14,1400,상품상세,gig,상품,3,41201,자기소개서,4,412.0,41201.0,문서작성,자기소개서·이력서,자기소개서
1,a6d462c3-d4a3-4a2c-8942-88f2cae0012e,com.kmong.kmong,8a871e50-0717-4aed-9bad-04ac3c3793be,2018-09-28 00:00:02+09:00,2018,9,28,0,0,2,Samsung,SM-N935S,Android,7.0,3.3.5,goal,gig,gig_detail,view,gig_detail.view,41201.0,unattributed,unattributed,NaN,NaN,NaN,False,False,14,1400,상품상세,gig,상품,3,41201,자기소개서,4,412.0,41201.0,문서작성,자기소개서·이력서,자기소개서
2,7d06d33e-cd1f-4064-a3d3-cb947424ff97,com.kmong.kmong,8a871e50-0717-4aed-9bad-04ac3c3793be,2018-09-28 00:00:03+09:00,2018,9,28,0,0,3,Samsung,SM-N935S,Android,7.0,3.3.5,goal,gig,gig_detail,view,gig_detail.view,41201.0,unattributed,unattributed,NaN,NaN,NaN,False,False,14,1400,상품상세,gig,상품,3,41201,자기소개서,4,412.0,41201.0,문서작성,자기소개서·이력서,자기소개서
3,497ae46f-6749-4094-853f-d5b0b33e18fc,com.kmong.kmong,8a871e50-0717-4aed-9bad-04ac3c3793be,2018-09-28 00:00:06+09:00,2018,9,28,0,0,6,Samsung,SM-N935S,Android,7.0,3.3.5,goal,gig,gig_detail,view,gig_detail.view,41201.0,unattributed,unattributed,NaN,NaN,NaN,False,False,14,1400,상품상세,gig,상품,3,41201,자기소개서,4,412.0,41201.0,문서작성,자기소개서·이력서,자기소개서
4,291c4d91-86c2-4102-b6c9-070bee255a4d,com.kmong.kmong,8a871e50-0717-4aed-9bad-04ac3c3793be,2018-09-28 00:00:28+09:00,2018,9,28,0,0,28,Samsung,SM-N935S,Android,7.0,3.3.5,goal,gig,gig_detail,view,gig_detail.view,41201.0,unattributed,unattributed,NaN,NaN,NaN,True,False,14,1400,상품상세,gig,상품,3,41201,자기소개서,4,412.0,41201.0,문서작성,자기소개서·이력서,자기소개서


<table border="1" class="dataframe" style="float: left;">
    <thead>
        <tr style="text-align: right;">
            <th></th>
            <th>row_uuid</th>
            <th>app_package_name</th>
            <th>user_id</th>
            <th>event_datetime</th>
            <th>event_datetime_year</th>
            <th>event_datetime_month</th>
            <th>event_datetime_day</th>
            <th>event_datetime_hour</th>
            <th>event_datetime_minute</th>
            <th>event_datetime_second</th>
            <th>device_manufacturer</th>
            <th>device_type</th>
            <th>os_type</th>
            <th>os_version</th>
            <th>app_version</th>
            <th>event_category</th>
            <th>view_category</th>
            <th>view_id</th>
            <th>view_action</th>
            <th>funnel_desc</th>
            <th>view_desc</th>
            <th>category_name</th>
            <th>category1</th>
            <th>category2</th>
            <th>category3</th>
            <th>channel</th>
            <th>params_campaign</th>
            <th>params_medium</th>
            <th>params_term</th>
            <th>is_first_activity</th>
            <th>is_first_goal_activity</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <th>0</th>
            <td>fd2a188c-bc9b-4702-9c47-b546b2614817</td>
            <td>com.kmong.iOS</td>
            <td>F36FAA62-ADAC-4AA5-9B00-1FD6CB7EE957</td>
            <td>2018-09-27 15:00:00</td>
            <td>2018</td>
            <td>9</td>
            <td>27</td>
            <td>15</td>
            <td>0</td>
            <td>0</td>
            <td>Apple</td>
            <td>iPhone</td>
            <td>iOS</td>
            <td>11.4.1</td>
            <td>4.0.4</td>
            <td>goal</td>
            <td>home</td>
            <td>home</td>
            <td>view</td>
            <td>홈</td>
            <td>홈 (탭)</td>
            <td>NaN</td>
            <td>NaN</td>
            <td>NaN</td>
            <td>NaN</td>
            <td>unattributed</td>
            <td>NaN</td>
            <td>NaN</td>
            <td>NaN</td>
            <td>False</td>
            <td>False</td>
        </tr>
        <tr>
            <th>1</th>
            <td>e62dccef-dd70-4415-8a33-c8324ddaed38</td>
            <td>com.kmong.kmong</td>
            <td>8a871e50-0717-4aed-9bad-04ac3c3793be</td>
            <td>2018-09-27 15:00:00</td>
            <td>2018</td>
            <td>9</td>
            <td>27</td>
            <td>15</td>
            <td>0</td>
            <td>0</td>
            <td>Samsung</td>
            <td>SM-N935S</td>
            <td>Android</td>
            <td>7.0</td>
            <td>3.3.5</td>
            <td>goal</td>
            <td>gig</td>
            <td>gig_detail</td>
            <td>view</td>
            <td>상품</td>
            <td>상품상세</td>
            <td>자기소개서</td>
            <td>문서작성</td>
            <td>자기소개서·이력서</td>
            <td>자기소개서</td>
            <td>unattributed</td>
            <td>NaN</td>
            <td>NaN</td>
            <td>NaN</td>
            <td>False</td>
            <td>False</td>
        </tr>
        <tr>
            <th>2</th>
            <td>14eb3197-db83-493a-b7be-83582960c40b</td>
            <td>com.kmong.iOS</td>
            <td>A9E5778A-8F3D-4597-9718-74BF953A9F64</td>
            <td>2018-09-27 15:00:00</td>
            <td>2018</td>
            <td>9</td>
            <td>27</td>
            <td>15</td>
            <td>0</td>
            <td>0</td>
            <td>Apple</td>
            <td>iPhone</td>
            <td>iOS</td>
            <td>12.0</td>
            <td>4.0.4</td>
            <td>goal</td>
            <td>inbox</td>
            <td>inbox_detail</td>
            <td>view</td>
            <td>메시지</td>
            <td>메시지목록-상세</td>
            <td>NaN</td>
            <td>NaN</td>
            <td>NaN</td>
            <td>NaN</td>
            <td>unattributed</td>
            <td>NaN</td>
            <td>NaN</td>
            <td>NaN</td>
            <td>False</td>
            <td>False</td>
        </tr>
        <tr>
            <th>3</th>
            <td>f9bb91af-248b-44dc-9f5c-1c00b37ea97b</td>
            <td>com.kmong.iOS</td>
            <td>168761CB-CB67-4592-867D-52780D651297</td>
            <td>2018-09-27 15:00:01</td>
            <td>2018</td>
            <td>9</td>
            <td>27</td>
            <td>15</td>
            <td>0</td>
            <td>1</td>
            <td>Apple</td>
            <td>iPhone</td>
            <td>iOS</td>
            <td>11.4.1</td>
            <td>4.0.4</td>
            <td>foreground</td>
            <td>NaN</td>
            <td>NaN</td>
            <td>NaN</td>
            <td>NaN</td>
            <td>NaN</td>
            <td>NaN</td>
            <td>NaN</td>
            <td>NaN</td>
            <td>NaN</td>
            <td>NaN</td>
            <td>NaN</td>
            <td>NaN</td>
            <td>NaN</td>
            <td>NaN</td>
            <td>NaN</td>
        </tr>
        <tr>
            <th>4</th>
            <td>236e9946-7801-4898-b609-06c8ab1139dc</td>
            <td>com.kmong.iOS</td>
            <td>ACABB7C0-4C76-413A-B314-E5D6DA0D0E5D</td>
            <td>2018-09-27 15:00:02</td>
            <td>2018</td>
            <td>9</td>
            <td>27</td>
            <td>15</td>
            <td>0</td>
            <td>2</td>
            <td>Apple</td>
            <td>iPhone</td>
            <td>iOS</td>
            <td>11.4.1</td>
            <td>4.0.4</td>
            <td>goal</td>
            <td>buyer</td>
            <td>buyer_order_track</td>
            <td>view</td>
            <td>거래관리</td>
            <td>메뉴목록-구매관리-거래메시지</td>
            <td>NaN</td>
            <td>NaN</td>
            <td>NaN</td>
            <td>NaN</td>
            <td>web</td>
            <td>NaN</td>
            <td>NaN</td>
            <td>NaN</td>
            <td>False</td>
            <td>False</td>
        </tr>
    </tbody>
</table>

In [59]:
cols=["in_app_event_category","in_app_event_label","source_type","funnel_name","depth","category_id","category1_id","category2_id","category3_id"]

In [60]:
cols

['in_app_event_category',
 'in_app_event_label',
 'source_type',
 'funnel_name',
 'depth',
 'category_id',
 'category1_id',
 'category2_id',
 'category3_id']

In [61]:
df2.head()

,row_uuid,app_package_name,user_id,event_datetime,event_datetime_year,event_datetime_month,event_datetime_day,event_datetime_hour,event_datetime_minute,event_datetime_second,device_manufacturer,device_type,os_type,os_version,app_version,event_category,view_category,view_id,view_action,in_app_event_category,in_app_event_label,source_type,channel,params_campaign,params_medium,params_term,is_first_activity,is_first_goal_activity,lv1,lv2,view_desc,funnel_name,funnel_desc,depth,category_id,category_name,category1_id,category2_id,category3_id,category1,category2,category3
0,e62dccef-dd70-4415-8a33-c8324ddaed38,com.kmong.kmong,8a871e50-0717-4aed-9bad-04ac3c3793be,2018-09-28 00:00:00+09:00,2018,9,28,0,0,0,Samsung,SM-N935S,Android,7.0,3.3.5,goal,gig,gig_detail,view,gig_detail.view,41201.0,unattributed,unattributed,NaN,NaN,NaN,False,False,14,1400,상품상세,gig,상품,3,41201,자기소개서,4,412.0,41201.0,문서작성,자기소개서·이력서,자기소개서
1,a6d462c3-d4a3-4a2c-8942-88f2cae0012e,com.kmong.kmong,8a871e50-0717-4aed-9bad-04ac3c3793be,2018-09-28 00:00:02+09:00,2018,9,28,0,0,2,Samsung,SM-N935S,Android,7.0,3.3.5,goal,gig,gig_detail,view,gig_detail.view,41201.0,unattributed,unattributed,NaN,NaN,NaN,False,False,14,1400,상품상세,gig,상품,3,41201,자기소개서,4,412.0,41201.0,문서작성,자기소개서·이력서,자기소개서
2,7d06d33e-cd1f-4064-a3d3-cb947424ff97,com.kmong.kmong,8a871e50-0717-4aed-9bad-04ac3c3793be,2018-09-28 00:00:03+09:00,2018,9,28,0,0,3,Samsung,SM-N935S,Android,7.0,3.3.5,goal,gig,gig_detail,view,gig_detail.view,41201.0,unattributed,unattributed,NaN,NaN,NaN,False,False,14,1400,상품상세,gig,상품,3,41201,자기소개서,4,412.0,41201.0,문서작성,자기소개서·이력서,자기소개서
3,497ae46f-6749-4094-853f-d5b0b33e18fc,com.kmong.kmong,8a871e50-0717-4aed-9bad-04ac3c3793be,2018-09-28 00:00:06+09:00,2018,9,28,0,0,6,Samsung,SM-N935S,Android,7.0,3.3.5,goal,gig,gig_detail,view,gig_detail.view,41201.0,unattributed,unattributed,NaN,NaN,NaN,False,False,14,1400,상품상세,gig,상품,3,41201,자기소개서,4,412.0,41201.0,문서작성,자기소개서·이력서,자기소개서
4,291c4d91-86c2-4102-b6c9-070bee255a4d,com.kmong.kmong,8a871e50-0717-4aed-9bad-04ac3c3793be,2018-09-28 00:00:28+09:00,2018,9,28,0,0,28,Samsung,SM-N935S,Android,7.0,3.3.5,goal,gig,gig_detail,view,gig_detail.view,41201.0,unattributed,unattributed,NaN,NaN,NaN,True,False,14,1400,상품상세,gig,상품,3,41201,자기소개서,4,412.0,41201.0,문서작성,자기소개서·이력서,자기소개서


In [62]:
df2=df2.drop(cols,axis=1).head()

In [63]:
len(df2.columns)

33

In [64]:
df2.set_index('row_uuid')

,app_package_name,user_id,event_datetime,event_datetime_year,event_datetime_month,event_datetime_day,event_datetime_hour,event_datetime_minute,event_datetime_second,device_manufacturer,device_type,os_type,os_version,app_version,event_category,view_category,view_id,view_action,channel,params_campaign,params_medium,params_term,is_first_activity,is_first_goal_activity,lv1,lv2,view_desc,funnel_desc,category_name,category1,category2,category3
row_uuid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
e62dccef-dd70-4415-8a33-c8324ddaed38,com.kmong.kmong,8a871e50-0717-4aed-9bad-04ac3c3793be,2018-09-28 00:00:00+09:00,2018,9,28,0,0,0,Samsung,SM-N935S,Android,7.0,3.3.5,goal,gig,gig_detail,view,unattributed,NaN,NaN,NaN,False,False,14,1400,상품상세,상품,자기소개서,문서작성,자기소개서·이력서,자기소개서
a6d462c3-d4a3-4a2c-8942-88f2cae0012e,com.kmong.kmong,8a871e50-0717-4aed-9bad-04ac3c3793be,2018-09-28 00:00:02+09:00,2018,9,28,0,0,2,Samsung,SM-N935S,Android,7.0,3.3.5,goal,gig,gig_detail,view,unattributed,NaN,NaN,NaN,False,False,14,1400,상품상세,상품,자기소개서,문서작성,자기소개서·이력서,자기소개서
7d06d33e-cd1f-4064-a3d3-cb947424ff97,com.kmong.kmong,8a871e50-0717-4aed-9bad-04ac3c3793be,2018-09-28 00:00:03+09:00,2018,9,28,0,0,3,Samsung,SM-N935S,Android,7.0,3.3.5,goal,gig,gig_detail,view,unattributed,NaN,NaN,NaN,False,False,14,1400,상품상세,상품,자기소개서,문서작성,자기소개서·이력서,자기소개서
497ae46f-6749-4094-853f-d5b0b33e18fc,com.kmong.kmong,8a871e50-0717-4aed-9bad-04ac3c3793be,2018-09-28 00:00:06+09:00,2018,9,28,0,0,6,Samsung,SM-N935S,Android,7.0,3.3.5,goal,gig,gig_detail,view,unattributed,NaN,NaN,NaN,False,False,14,1400,상품상세,상품,자기소개서,문서작성,자기소개서·이력서,자기소개서
291c4d91-86c2-4102-b6c9-070bee255a4d,com.kmong.kmong,8a871e50-0717-4aed-9bad-04ac3c3793be,2018-09-28 00:00:28+09:00,2018,9,28,0,0,28,Samsung,SM-N935S,Android,7.0,3.3.5,goal,gig,gig_detail,view,unattributed,NaN,NaN,NaN,True,False,14,1400,상품상세,상품,자기소개서,문서작성,자기소개서·이력서,자기소개서
